In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

Task1 
1. Load CIFAR1-10 dataset

In [9]:
# 1. Load CIFAR-10 Dataset
transform = transforms.ToTensor()
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, activation_fn=nn.LeakyReLU()):
        super(SimpleCNN, self).__init__()
        # The first layer has 3 input channels and 32 output channels. The point here is
        # that the CIFR images are 32 * 32, and RGB channels after the to tensor. So, we take
        self.conv_layer = nn.Conv2d(3, 32, kernel_size=5) 
        # Fully connected layer - the hidden layers are 28 * 28 hence and pooling reduces it by 2
        # Hence, 14 * 14 and mult that with 32 feature/conv layers.
        self.nn_layer = nn.Linear(32 * 14 * 14, 10)
        self.activation = activation_fn  # Activation function
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
    
    def forward(self, x):
        x = self.pool(self.activation(self.conv_layer(x))) 
        x = torch.flatten(x, 1)  # Flatten the feature maps, as the next layer is 1d layer
        x = self.nn_layer(x)  # Output layer
        return x

# ANN-1 excercise
def train_model(model, optimizer, error_func, epochs=10, log_dir="", device="cpu"):
    writer = SummaryWriter(log_dir)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in trainloader:
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = error_func(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total  # Compute accuracy
        writer.add_scalar('Loss/train', running_loss / len(trainloader), epoch)  # Log loss
        writer.add_scalar('Accuracy/train', train_acc, epoch)  # Log accuracy
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}, Accuracy: {train_acc}%')
    writer.close()

# Train model with LeakyReLU and SGD optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device I have is {device}")

model1 = SimpleCNN(nn.LeakyReLU())
optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model1, optimizer1, err_func, log_dir='logs/leakyrelu_sgd')

# Train model with LeakyReLU and Adam optimizer
model2 = SimpleCNN(nn.LeakyReLU())
optimizer2 = optim.Adam(model2.parameters())
train_model(model2, optimizer2, err_func, log_dir='logs/leakyrelu_adam')

# Train model with Tanh and Adam optimizer
model3 = SimpleCNN(nn.Tanh())
optimizer3 = optim.Adam(model3.parameters())
train_model(model3, optimizer3, err_func, log_dir='logs/tanh_adam')



The device I have is cuda
Epoch 1, Loss: 2.2973031162300988, Accuracy: 11.396%
Epoch 2, Loss: 2.273641569535141, Accuracy: 15.694%
Epoch 3, Loss: 2.2523760082166824, Accuracy: 18.442%
